# Exercice 1 : Initialisation de l’environnement et des structures de données

In [1]:
import gymnasium as gym
import numpy as np

# Initialisation de l’environnement
env = gym.make("Taxi-v3")

# Nombre d’états et d’actions
state_size = env.observation_space.n
action_size = env.action_space.n

print(f"Nombre d'états: {state_size}")
print(f"Nombre d'actions: {action_size}")

# Création de la table de politique avec probabilité uniforme pour chaque action
policy_table = np.ones((state_size, action_size)) / action_size

# Création de la table de valeurs initialisée à zéro
value_table = np.zeros(state_size)

# Affichage des premières lignes des tables
print("\nPremières lignes de la table de politique:")
print(policy_table[:5])

print("\nPremières valeurs de la table de valeur:")
print(value_table[:5])

Nombre d'états: 500
Nombre d'actions: 6

Premières lignes de la table de politique:
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

Premières valeurs de la table de valeur:
[0. 0. 0. 0. 0.]


# Exercice 2: Exploration et collecte d'épisodes

In [4]:
# Nombre d'épisodes
num_episodes = 20

for episode in range(num_episodes):
    state, _ = env.reset()  # Initialisation de l'état
    total_reward = 0

    print(f"\nÉpisode {episode + 1} :")

    for t in range(20):  # Exécuter 20 étapes par épisode
        action = env.action_space.sample()  # Choix d'une action aléatoire
        next_state, reward, done, truncated, info = env.step(action)
        total_reward += reward

        print(f"Étape {t + 1} - Action: {action}, Récompense: {reward}")

        if done:  # Si l'épisode se termine avant 20 étapes
            break

    print(f"Récompense totale de l'épisode: {total_reward}")

env.close()


Épisode 1 :
Étape 1 - Action: 4, Récompense: -10
Étape 2 - Action: 3, Récompense: -1
Étape 3 - Action: 5, Récompense: -10
Étape 4 - Action: 1, Récompense: -1
Étape 5 - Action: 5, Récompense: -10
Étape 6 - Action: 3, Récompense: -1
Étape 7 - Action: 4, Récompense: -10
Étape 8 - Action: 4, Récompense: -10
Étape 9 - Action: 0, Récompense: -1
Étape 10 - Action: 0, Récompense: -1
Étape 11 - Action: 2, Récompense: -1
Étape 12 - Action: 4, Récompense: -10
Étape 13 - Action: 5, Récompense: -10
Étape 14 - Action: 3, Récompense: -1
Étape 15 - Action: 5, Récompense: -10
Étape 16 - Action: 2, Récompense: -1
Étape 17 - Action: 1, Récompense: -1
Étape 18 - Action: 5, Récompense: -10
Étape 19 - Action: 0, Récompense: -1
Étape 20 - Action: 4, Récompense: -10
Récompense totale de l'épisode: -110

Épisode 2 :
Étape 1 - Action: 5, Récompense: -10
Étape 2 - Action: 4, Récompense: -10
Étape 3 - Action: 3, Récompense: -1
Étape 4 - Action: 4, Récompense: -10
Étape 5 - Action: 2, Récompense: -1
Étape 6 - Act

# Exercice 3: Mise a jour de la politique avec PPO

In [7]:
import numpy as np

# Hyperparamètres
gamma = 0.99  # Facteur d'actualisation des récompenses futures
lr_policy = 0.1  # Taux d'apprentissage pour la politique
lr_value = 0.1  # Taux d'apprentissage pour la fonction de valeur
clip_epsilon = 0.2  # Seuil de clipping



def compute_discounted_rewards(rewards, gamma):
    """Calcule les récompenses cumulées (discounted rewards)."""
    discounted_rewards = np.zeros_like(rewards, dtype=np.float32)
    cumulative_reward = 0
    for t in reversed(range(len(rewards))):
        cumulative_reward = rewards[t] + gamma * cumulative_reward
        discounted_rewards[t] = cumulative_reward
    return discounted_rewards

def update_policy_value(policy_table, value_table, states, actions, rewards):
    """Met à jour la politique et la fonction de valeur avec PPO."""
    # Calcul des récompenses cumulées
    discounted_rewards = compute_discounted_rewards(rewards, gamma)

    for t in range(len(states)):
        state, action, reward = states[t], actions[t], rewards[t]
        
        # Calcul de l'avantage At = Rt - V(st)
        advantage = discounted_rewards[t] - value_table[state]

        # Mise à jour de la fonction de valeur avec une learning rate
        value_table[state] += lr_value * advantage

        # Mise à jour de la politique avec clipping
        old_prob = policy_table[state, action]
        new_prob = old_prob * np.exp(lr_policy * advantage)  # Mise à jour proportionnelle
        new_prob = np.clip(new_prob, old_prob * (1 - clip_epsilon), old_prob * (1 + clip_epsilon))  # Clipping
        
        # Normalisation de la distribution des probabilités
        policy_table[state] = policy_table[state] / np.sum(policy_table[state])  # Normalisation
        policy_table[state, action] = new_prob  # Mise à jour de la probabilité de l'action sélectionnée

    return policy_table, value_table

# Exemple d'utilisation
episode_states = [0, 1, 2, 3]  # Liste des états visités (exemple)
episode_actions = [1, 2, 0, 3]  # Liste des actions prises
episode_rewards = [10, -5, 2, 8]  # Liste des récompenses obtenues

# Mise à jour de la politique et des valeurs
policy_table, value_table = update_policy_value(policy_table, value_table, episode_states, episode_actions, episode_rewards)

print("Table de politique mise à jour (extrait) :", policy_table[:5])
print("Table de valeurs mise à jour (extrait) :", value_table[:5])

Table de politique mise à jour (extrait) : [[0.16666667 0.2        0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.2        0.16666667 0.16666667 0.16666667]
 [0.2        0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.2        0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]
Table de valeurs mise à jour (extrait) : [1.47725916 0.48207998 0.99200001 0.8        0.        ]


# Exercice 4 : Évaluation de l’agent après entraînement

In [9]:
num_eval_episodes = 20
total_rewards = []
max_steps = 100  # Prevents infinite loops

for ep in range(num_eval_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    steps = 0  # Step counter

    while not done and steps < max_steps:
        # Normalize policy before selecting action (prevents NaNs)
        action_probs = policy_table[state] / np.sum(policy_table[state] + 1e-8)  
        action = np.argmax(action_probs)  

        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        state = next_state
        steps += 1  # Increase step count

    total_rewards.append(total_reward)
    print(f"Épisode {ep + 1}, Récompense totale: {total_reward}")

# Avoid infinite mean if total_rewards is empty
mean_reward = np.mean(total_rewards) if total_rewards else 0
print("Récompense moyenne après entraînement:", mean_reward)

Épisode 1, Récompense totale: -100
Épisode 2, Récompense totale: -100
Épisode 3, Récompense totale: -100
Épisode 4, Récompense totale: -100
Épisode 5, Récompense totale: -100
Épisode 6, Récompense totale: -100
Épisode 7, Récompense totale: -100
Épisode 8, Récompense totale: -100
Épisode 9, Récompense totale: -100
Épisode 10, Récompense totale: -100
Épisode 11, Récompense totale: -100
Épisode 12, Récompense totale: -100
Épisode 13, Récompense totale: -100
Épisode 14, Récompense totale: -100
Épisode 15, Récompense totale: -100
Épisode 16, Récompense totale: -100
Épisode 17, Récompense totale: -100
Épisode 18, Récompense totale: -100
Épisode 19, Récompense totale: -100
Épisode 20, Récompense totale: -100
Récompense moyenne après entraînement: -100.0
